In [4]:
import chromadb
import fitz
import spacy
from sentence_transformers import SentenceTransformer
from pprint import pprint
from pathlib import Path
import os
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_openai import OpenAIEmbeddings
import openai
from pymongo import MongoClient
from chromadb import Client as ChromaClient


In [5]:
#Listando arquivos da pasta local "pdf_documents"

pdf_folder_path = "../pdf_documents"
pdf_files = [f for f in os.listdir(pdf_folder_path) if f.endswith(".pdf")]

print("Arquivos encontrados:")
for pdf in pdf_files:
    print(f"- {pdf}")

Arquivos encontrados:
- EIA_example1.pdf
- RIMA_example1.pdf
- EIA_Example2.pdf
- RIMA_Example2.pdf
- EIA_Example3.pdf
- RIMA_Example3.pdf


In [6]:
import fitz
import re
import spacy

# Carrega o modelo e aumenta o limite máximo de caracteres
nlp = spacy.load("pt_core_news_sm")
nlp.max_length = 2_000_000  # Aumentado para lidar com documentos grandes

# Palavras e frases a ignorar
GENERIC_IGNORE_KEYWORDS = {
    "sumário", "índice", "resumo", "anexo", "figura", "tabela", "referência", "bibliografia",
    "conteúdo", "protocolo", "gov", "secretaria", "assinatura", "documento assinado",
    "orientações gerais", "preenchimento", "manual", "parecer técnico"
}

PHRASE_BLACKLIST = [
    r"monitoramento de.*\(e-cenários\)",
    r"^página \d+.*",
    r"^mapas\s*-\s*",
    r"^inserção do estudo.*",
    r"^um arquivo não substitui.*",
    r"^o link para.*tipologia.*",
    r"documentos, manifestações.*",
    r"requerimento.*preenchido.*",
    r"documento.*válido.*"
]

KEY_VERBS = {
    "avaliar", "caracterizar", "delimitar", "analisar", "estimar",
    "descrever", "propor", "identificar", "considerar", "indicar",
    "demonstrar", "impactar", "recomendar", "quantificar"
}

def contains_key_verb(sent):
    return any(tok.lemma_ in KEY_VERBS for tok in nlp(sent))

def is_line_irrelevant(line):
    lower = line.lower()

    if len(lower.strip()) < 15:
        return True
    if any(k in lower for k in GENERIC_IGNORE_KEYWORDS):
        return True
    if any(re.search(pattern, lower) for pattern in PHRASE_BLACKLIST):
        return True
    if re.search(r"\d{2}/\d{2}/\d{2,4}", lower):
        return True
    if lower.endswith(".pdf"):
        return True
    if re.match(r"^\d+(\.\d+)+\s+", lower):
        return True
    if lower.isupper() and len(lower.split()) > 5:
        return True
    return False

# Função para dividir o texto em blocos menores
def split_text(text, max_chars=500000):
    chunks = []
    while len(text) > max_chars:
        split_point = text[:max_chars].rfind(".") + 1  # tenta dividir no ponto final
        if split_point < 50:
            split_point = max_chars  # se não encontrar ponto, quebra direto
        chunks.append(text[:split_point].strip())
        text = text[split_point:].strip()
    if text:
        chunks.append(text)
    return chunks

def extract_relevant_sentences(text, min_words=4):
    relevant = []
    chunks = split_text(text)
    for chunk in chunks:
        doc = nlp(chunk)
        for sent in doc.sents:
            s = sent.text.strip()
            if len(s.split()) < min_words:
                continue
            if contains_key_verb(s):
                relevant.append(s)
            elif any(tok.pos_ == "VERB" for tok in sent):
                relevant.append(s)
    return relevant

def extract_clean_text_with_spacy(pdf_path):
    try:
        with fitz.open(pdf_path) as doc:
            raw_text = "\n".join([page.get_text("text") for page in doc])

        lines = raw_text.splitlines()
        filtered = [line.strip() for line in lines if not is_line_irrelevant(line.strip())]
        cleaned_text = " ".join(filtered)

        relevant_sentences = extract_relevant_sentences(cleaned_text)
        return "\n".join(relevant_sentences) if relevant_sentences else None

    except Exception as e:
        print(f"Erro ao processar {pdf_path}: {e}")
        return None


In [7]:
# Exemplo de funcionamento da função com o primeiro arquivo da pasta
# if pdf_files:
#     pdf_path = os.path.join(pdf_folder_path, pdf_files[3]) 
#     pdf_text = extract_clean_text_with_spacy(pdf_path)  
#     print("Exemplo de texto extraído do primeiro arquivo da pasta")
#     print(f"\n Nome do arquivo : {pdf_files[3]}\n")
#     print(pdf_text)
# else:
#     print("Lista de pdf_files vazia")


In [8]:
#Extraindo o texto de todos os arquivos da pasta

pdf_texts = {}

for pdf_file in pdf_files:
    pdf_path = os.path.join(pdf_folder_path, pdf_file)

    print(f"Processando: {pdf_file}...")  # Mostra qual arquivo está sendo lido

    try:
        pdf_texts[pdf_file] = extract_clean_text_with_spacy(pdf_path)  
        print(f"Extração concluída: {pdf_file}\n")
    except Exception as e:
        print(f"Erro ao processar {pdf_file}: {e}\n")

print(f"\nOs textos de {len(pdf_texts)} documentos foram extraídos com sucesso.")


Processando: EIA_example1.pdf...
Extração concluída: EIA_example1.pdf

Processando: RIMA_example1.pdf...
Extração concluída: RIMA_example1.pdf

Processando: EIA_Example2.pdf...
MuPDF error: library error: zlib error: invalid distance too far back

Extração concluída: EIA_Example2.pdf

Processando: RIMA_Example2.pdf...
Extração concluída: RIMA_Example2.pdf

Processando: EIA_Example3.pdf...
Extração concluída: EIA_Example3.pdf

Processando: RIMA_Example3.pdf...
Extração concluída: RIMA_Example3.pdf


Os textos de 6 documentos foram extraídos com sucesso.


In [9]:
# Carregar a chave da API do ambiente
openai_api_key = os.getenv("OPENAI_API_KEY")

In [10]:

#Função para dividir os textos em chunks
def split_text_into_chunks(text, chunk_size=512, chunk_overlap=50):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
    )
    
    chunks = splitter.split_text(text)
    return chunks

In [11]:
#Função para gerar os embeddings de cada chunk de acordo com a formtação da OPENAI
def generate_embeddings(chunks):
    embeddings_model = OpenAIEmbeddings(openai_api_key=openai_api_key)
    return embeddings_model.embed_documents(chunks)

In [12]:
#Gerando os chunks e embeedings para os textos extraídos dos documentos
text_chunks = {}
embeddings = {}

for pdf_file, full_text in pdf_texts.items():
    if not isinstance(full_text, str):  
        print(f"Aviso: O texto extraído de {pdf_file} não é uma string. Convertendo para string vazia.")
        full_text = ""
    
    text_chunks[pdf_file] = split_text_into_chunks(full_text)
    embeddings[pdf_file] = generate_embeddings(text_chunks[pdf_file])
    print(f"Texto do arquivo {pdf_file} dividido em {len(text_chunks[pdf_file])} blocos e embeddings gerados.")

print(f"\nOs textos de {len(pdf_texts)} documentos foram particionados e processados em embeddings.")


Texto do arquivo EIA_example1.pdf dividido em 573 blocos e embeddings gerados.
Texto do arquivo RIMA_example1.pdf dividido em 61 blocos e embeddings gerados.
Texto do arquivo EIA_Example2.pdf dividido em 1874 blocos e embeddings gerados.
Texto do arquivo RIMA_Example2.pdf dividido em 178 blocos e embeddings gerados.
Texto do arquivo EIA_Example3.pdf dividido em 1974 blocos e embeddings gerados.
Texto do arquivo RIMA_Example3.pdf dividido em 83 blocos e embeddings gerados.

Os textos de 6 documentos foram particionados e processados em embeddings.


In [13]:
# Configurando o ChromaDB com persistência
chroma_client = chromadb.PersistentClient(path="./chroma_db")

In [14]:
# Conexões
mongo_client = MongoClient("mongodb://localhost:27017")
mongo_db = mongo_client["rag_db"]
mongo_chunks = mongo_db["chunks"]

collection = chroma_client.get_or_create_collection(name="rag_embeddings")

In [15]:
#salvando dados processados no banco de dados

total_chunks = 0

for pdf_file in text_chunks:
    chunks = text_chunks[pdf_file]
    vector_list = embeddings[pdf_file]

    print(f"Salvando documento: {pdf_file} ({len(chunks)} chunks)")

    for idx, (chunk_text, vector) in enumerate(zip(chunks, vector_list)):
        chunk_id = f"{pdf_file}_chunk_{idx}"

        # Verificar se o chunk já existe no MongoDB
        if mongo_chunks.find_one({"chunk_id": chunk_id}):
            print(f" - Chunk '{chunk_id}' já existe no MongoDB. Pulando.")
            continue

        # Verificar se o ID já existe no ChromaDB (opcional, pode ser custoso)
        existing_ids = collection.get(ids=[chunk_id])
        if existing_ids and existing_ids["ids"]:
            print(f" - Chunk '{chunk_id}' já existe no ChromaDB. Pulando.")
            continue

        # 1. Inserir no MongoDB
        mongo_chunks.insert_one({
            "pdf_file": pdf_file,
            "chunk_index": idx,
            "chunk_text": chunk_text,
            "chunk_id": chunk_id
        })

        # 2. Inserir no ChromaDB
        collection.add(
            ids=[chunk_id],
            embeddings=[vector],
            metadatas=[{
                "chunk_index": idx,
                "source_file": pdf_file
            }]
        )

        total_chunks += 1

    print(f" -> Documento '{pdf_file}' salvo com sucesso.\n")

print(f"\n{total_chunks} chunks armazenados no MongoDB e no ChromaDB.")


Salvando documento: EIA_example1.pdf (573 chunks)
 - Chunk 'EIA_example1.pdf_chunk_0' já existe no MongoDB. Pulando.
 - Chunk 'EIA_example1.pdf_chunk_1' já existe no MongoDB. Pulando.
 - Chunk 'EIA_example1.pdf_chunk_2' já existe no MongoDB. Pulando.
 - Chunk 'EIA_example1.pdf_chunk_3' já existe no MongoDB. Pulando.
 - Chunk 'EIA_example1.pdf_chunk_4' já existe no MongoDB. Pulando.
 - Chunk 'EIA_example1.pdf_chunk_5' já existe no MongoDB. Pulando.
 - Chunk 'EIA_example1.pdf_chunk_6' já existe no MongoDB. Pulando.
 - Chunk 'EIA_example1.pdf_chunk_7' já existe no MongoDB. Pulando.
 - Chunk 'EIA_example1.pdf_chunk_8' já existe no MongoDB. Pulando.
 - Chunk 'EIA_example1.pdf_chunk_9' já existe no MongoDB. Pulando.
 - Chunk 'EIA_example1.pdf_chunk_10' já existe no MongoDB. Pulando.
 - Chunk 'EIA_example1.pdf_chunk_11' já existe no MongoDB. Pulando.
 - Chunk 'EIA_example1.pdf_chunk_12' já existe no MongoDB. Pulando.
 - Chunk 'EIA_example1.pdf_chunk_13' já existe no MongoDB. Pulando.
 - Chunk

In [16]:
pipeline = [
    {"$group": {
        "_id": "$pdf_file",
        "total_chunks": {"$sum": 1}
    }}
]

resultados = list(mongo_chunks.aggregate(pipeline))

print("Arquivos no MongoDB:")
for doc in resultados:
    print(f"- {doc['_id']}: {doc['total_chunks']} chunks")


Arquivos no MongoDB:
- EIA_Example2.pdf: 1874 chunks
- EIA_example1.pdf: 592 chunks
- RIMA_example1.pdf: 63 chunks
- EIA_Example3.pdf: 1974 chunks
- RIMA_Example3.pdf: 83 chunks
- RIMA_Example2.pdf: 178 chunks


In [17]:
chroma_data = collection.get()

print("Dados armazenados no ChromaDB:")
for chunk_id, metadata in zip(chroma_data["ids"], chroma_data["metadatas"]):
    print(f"- {chunk_id} (arquivo: {metadata['source_file']}, chunk: {metadata['chunk_index']})")


Dados armazenados no ChromaDB:
- EIA_example1.pdf_chunk_0 (arquivo: EIA_example1.pdf, chunk: 0)
- EIA_example1.pdf_chunk_1 (arquivo: EIA_example1.pdf, chunk: 1)
- EIA_example1.pdf_chunk_2 (arquivo: EIA_example1.pdf, chunk: 2)
- EIA_example1.pdf_chunk_3 (arquivo: EIA_example1.pdf, chunk: 3)
- EIA_example1.pdf_chunk_4 (arquivo: EIA_example1.pdf, chunk: 4)
- EIA_example1.pdf_chunk_5 (arquivo: EIA_example1.pdf, chunk: 5)
- EIA_example1.pdf_chunk_6 (arquivo: EIA_example1.pdf, chunk: 6)
- EIA_example1.pdf_chunk_7 (arquivo: EIA_example1.pdf, chunk: 7)
- EIA_example1.pdf_chunk_8 (arquivo: EIA_example1.pdf, chunk: 8)
- EIA_example1.pdf_chunk_9 (arquivo: EIA_example1.pdf, chunk: 9)
- EIA_example1.pdf_chunk_10 (arquivo: EIA_example1.pdf, chunk: 10)
- EIA_example1.pdf_chunk_11 (arquivo: EIA_example1.pdf, chunk: 11)
- EIA_example1.pdf_chunk_12 (arquivo: EIA_example1.pdf, chunk: 12)
- EIA_example1.pdf_chunk_13 (arquivo: EIA_example1.pdf, chunk: 13)
- EIA_example1.pdf_chunk_14 (arquivo: EIA_example1.

In [ ]:

#estudar leitura de imagens (leitura de mapas)
